# Download Chinese Celebrity Face Images and Check for Image Quality

### In this file, we use a library called simple image download to download celebrity face images. 
### Then we make an s3 bucket to store these images.
### Then, we filter these images using Rekognition's detect faces method in order to cull low-quality images (i.e., images where multiple faces are detected, images that are too small, etc.)

In [7]:
# Let us install the library we are going to use to download images:
! pip install simple_image_download # Python library that simplifies the process of downloading images from the internet
! pip install scikit-image # collection of algorithms for image processing

INFO: pip is looking at multiple versions of simple-image-download to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 67.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.9/222.9 kB 42.6 MB/s eta 0:00:00


In [8]:
import pandas as pd # library for data manipulation
import boto3 # Amazon Web Services (AWS) Software Development Kit (SDK) for Python; used to interact with AWS services, such as S3 (Simple Storage Service),
import json # imports the built-in json module, which provides methods for encoding and decoding JSON data
import s3fs # imports the s3fs library, which is a Python interface to interact with data stored on Amazon S3
import io # imports the built-in io module, which provides the Python interfaces for stream handling.
import os

import numpy as np
from IPython.display import Image as IImage

from PIL import Image, ImageDraw, ExifTags, ImageColor
from matplotlib.pyplot import imshow


In [43]:
print(f"Boto3 Version: {boto3.__version__}")
! pip install --upgrade boto3


Boto3 Version: 1.28.80
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 42.1 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.80
    Uninstalling botocore-1.31.80:
      Successfully uninstalled botocore-1.31.80
  Attempting uninstall: boto3
    Found existing installation: boto3 1.28.80
    Uninstalling boto3-1.28.80:
      Successfully uninstalled boto3-1.28.80
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.80 requires botocore==1.31.80, but you have botocore 1.32.6 which is incompatible.


In [3]:
# Please download these CSV files from the github 

# Link to the github: https://github.com/sophiaxxiao/Celebrity_recognition_project.git

# You can type in the terminal 'git clone https://github.com/sophiaxxiao/Celebrity_recognition_project.git' to obtain these files


# Make sure you have the correct paths 
chinese_celebs = pd.read_csv('/home/ec2-user/SageMaker/Celebrity_recognition_project/chinese_celebs.csv')

western_celebs = pd.read_csv('/home/ec2-user/SageMaker/Celebrity_recognition_project/western_celebs.csv')


In [4]:
chinese_celebs

,Rank,Name,Earnings,Occupation,Country
0,1,Fan Bingbing,300 million yuan,Actress,China
1,2,Lu Han,210 million yuan,Actor and Singer,China
2,3,Yang Mi,200 million yuan,Actress,China
3,4,Zhao Liying,190 million yuan,Actress,China
4,5,Yang Yang,240 million yuan,Actor,China
...,...,...,...,...,...
73,75,Xie Na,40 million yuan,TV Host and Actress,China
74,76,Hawick Lau,75 million yuan,Actor,China
75,78,Huang Xuan,50 million yuan,Actor,China
76,79,Ma Sichun,40 million yuan,Actress,China


In [5]:
western_celebs

,Rank,Name,Earnings,Occupation,Country
0,1,Diddy,$130 million,Musician,U.S.
1,2,Beyoncé,$105 million,Musician,U.S.
2,3,J.K. Rowling,$95 million,Author,U.K.
3,4,Drake,$94 million,Musician,Canada
4,6,The Weeknd,$92 million,Musician,Canada
...,...,...,...,...,...
75,95,Chance The Rapper,$33 million,Musician,U.S.
76,95,Katy Perry,$33 million,Musician,U.S.
77,97,Carmelo Anthony,$32.6 million,Athlete,U.S.
78,98,Jason Aldean,$32.5 million,Musician,U.S.


In [6]:
from simple_image_download import simple_image_download as simp 

chinese_celebrity_names = chinese_celebs['Name'][0:14] # selecting the top 14 celebrities 

response = simp.simple_image_download

# The code below downloads the images
for name in chinese_celebrity_names:
 response().download(name , 50)

HTTPSConnectionPool(host='media2.malaymail.com', port=443): Max retries exceeded with url: /uploads/articles/2020/2020-06/luhan0506.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f20900e17e0>, 'Connection to media2.malaymail.com timed out. (connect timeout=None)'))
HTTPSConnectionPool(host='pcwww.liv.ac.uk', port=443): Max retries exceeded with url: /~hanlu/LuHan2021.jpg (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))
HTTPSConnectionPool(host='thelosangelestribune.com', port=443): Max retries exceeded with url: /wp-content/uploads/2023/04/Jay-Chou-20230328-3-683x1024.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f20900e2a10>, 'Connection to thelosangelestribune.com timed out. (connect timeout=None)'))


KeyboardInterrupt: 

In [7]:
chinese_celebrity_names = chinese_celebs['Name'][15:29] # selecting next 14 celebrities 

response = simp.simple_image_download

# The code below downloads the images
for name in chinese_celebrity_names:
 response().download(name , 50)

HTTPSConnectionPool(host='www.euphoriazine.com', port=443): Max retries exceeded with url: /wp-content/uploads/2020/09/Courtesy-of-Zhang-Yixing-Studio-4.jpg (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)')))


In [8]:
chinese_celebrity_names = chinese_celebs['Name'][30:44] # selecting next 14 celebrities 

response = simp.simple_image_download

# The code below downloads the images
for name in chinese_celebrity_names:
 response().download(name , 50)

HTTPSConnectionPool(host='nhlofficials.com', port=443): Max retries exceeded with url: /wp-content/uploads/2020/09/Chris-Lee.png (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f208ae26920>, 'Connection to nhlofficials.com timed out. (connect timeout=None)'))
HTTPSConnectionPool(host='asianfilmfestival.us', port=443): Max retries exceeded with url: /wp-content/uploads/2019/09/Mark-Chao-090819-001.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f208ae27bb0>, 'Connection to asianfilmfestival.us timed out. (connect timeout=None)'))


In [9]:
chinese_celebrity_names = chinese_celebs['Name'][45:50] # selecting next 14 celebrities 

response = simp.simple_image_download

# The code below downloads the images
for name in chinese_celebrity_names:
 response().download(name , 50)

KeyboardInterrupt: 

In [11]:
!ls simple_images

Andy Lau      Huang Bo	      Kris Wu	    Lu Yi	  Xue Zhiqian
Angelababy    Huang Xiaoming  Li Bingbing   Mark Chao	  Yang Mi
Chen Xuedong  Huang Zitao     Lin Chi-Ling  Sun Li	  Yang Yang
Chris Lee     Hu Ge	      Lin Gengxin   Tang Yan	  Zhang Yixing
Deng Chao     Jackie Chan     Liu Shishi    Tong Liya	  Zhao Liying
Dilraba       Jay Chou	      Liu Tao	    Wallace Huo   Zhao Wei
Fan Bingbing  Jia Nailiang    Liu Yifei     Wang Kai	  Zheng Shuang
Faye Wong     Jing Boran      Li Yifeng     William Chan
He Jiong      Jing Tian       Lu Han	    Wu Xiubo


In [12]:
chinese_celeb_bucket = 'chinese-celebs' # (you need to change the name of the bucket -- make sure your bucket name is unique)

# Use this commmand to make the bucket 
!aws s3 mb s3://$chinese_celeb_bucket

# This is the AWS CLI command to create an S3 bucket. 
# The mb stands for "make bucket," and it's used to create a new S3 bucket.
# 's3://asian-faces-bucket' indicates that we're making our bucket in s3, and the part after the '//' is the name of the bucket


make_bucket: chinese-celebs


In [13]:
%%capture
foldername = 'simple_images' # make sure you change this string to the name of the folder where your images are located 

!ls $foldername 

!aws s3 mv ./$foldername s3://$chinese_celeb_bucket/ --recursive # move the contents of the folder into the s3 bucket


In [18]:
chinese_celebrity_names = chinese_celebs['Name'][46:50] # selecting the last few celebrities 

response = simp.simple_image_download

# The code below downloads the images
for name in chinese_celebrity_names:
 response().download(name , 50)

In [20]:
!aws s3 mv ./$foldername s3://$chinese_celeb_bucket/ --recursive # move the contents of the folder into the s3 bucket

In [21]:
client=boto3.client('rekognition')
import subprocess
import re
import s3fs 

In [ ]:
import boto3
from PIL import Image
from io import BytesIO

# Initialize the S3 client
s3 = boto3.client('s3')
fs = s3fs.S3FileSystem(anon=False)

bucket_name = 'chinese-celebs'

# List objects in the bucket
response = s3.list_objects_v2(Bucket=bucket_name)

# Initialize Rekognition client
rekognition_client = boto3.client('rekognition')

# Function to remove photo based on your criteria
def should_remove_photo(photo, bucket):
    # Perform label detection using Rekognition
    try:
        response = rekognition_client.detect_faces(Image={'S3Object': {'Bucket': bucket, 'Name': photo}})
    except rekognition_client.exceptions.InvalidImageFormatException as e:
        print(f"Invalid image format: {e}")
        return True
    # print(response)
    # Perform image checks
    if response['FaceDetails'] == []:
        return True
    
    image = Image.open(BytesIO(s3.get_object(Bucket=bucket, Key=photo)['Body'].read()))
    
    imgWidth, imgHeight = image.size

    face_details = response['FaceDetails'][0] if 'FaceDetails' in response and response['FaceDetails'] else None
    pitch = face_details['Pose']['Pitch'] if 'Pose' in face_details and 'Pitch' in face_details['Pose'] else None
    yaw = face_details['Pose']['Yaw'] if 'Pose' in face_details and 'Yaw' in face_details['Pose'] else None
    sunglasses = face_details.get('Sunglasses', {}).get('Value', None)
    bounding_box = face_details.get('BoundingBox', {})
    width = imgWidth * bounding_box.get('Width', 0)
    height = imgHeight * bounding_box.get('Height', 0)
    eyes_open = face_details.get('EyesOpen', {}).get('Value', None)
    number_faces_detected = len(response['FaceDetails'])
    occlusion_confidence = response['FaceDetails'][0].get('Occlusion', {}).get('Value', 0)

    
    # Check if Emotions array is present and non-empty
    emotions = face_details.get('Emotions', [])
    emotion = emotions[0]['Type'] if emotions else None
    emotion_confidence = emotions[0]['Confidence'] if emotions else None
    
    mouth_open = face_details.get('MouthOpen', {}).get('Value', None)
    
    if number_faces_detected > 1:
        print('Multiple faces detected')
        return True
    if width < 50 or height < 50:
        print('Too small')
        return True
    if abs(yaw) > 45:
        print('yaw too much')
        return True
    if pitch > 60 or pitch < -60:
        print('pitch too severe')
        return True
    if occulsion_confidence > 70:
        print('face occluded')
        return True
    return False


In [52]:
# Iterate through the objects in the bucket
for obj in response.get('Contents', [])[:500]:
    # Get the object key (file path)
    object_key = obj['Key']
    print(object_key)

    # Apply your function to the object
    if should_remove_photo(object_key, bucket_name):
        # Remove the file from the bucket
        s3.delete_object(Bucket=bucket_name, Key=object_key)
        
        print("Photo removed.")
    else:
        print("Keeping photo:", object_key)


Andy Lau/Andy Lau_11.jpg
Keeping photo: Andy Lau/Andy Lau_11.jpg
Andy Lau/Andy Lau_13.jpg
Keeping photo: Andy Lau/Andy Lau_13.jpg
Andy Lau/Andy Lau_14.jpg
Keeping photo: Andy Lau/Andy Lau_14.jpg
Andy Lau/Andy Lau_15.jpg
Keeping photo: Andy Lau/Andy Lau_15.jpg
Andy Lau/Andy Lau_17.jpg
Keeping photo: Andy Lau/Andy Lau_17.jpg
Andy Lau/Andy Lau_19.jpg
Keeping photo: Andy Lau/Andy Lau_19.jpg
Andy Lau/Andy Lau_20.jpg
Keeping photo: Andy Lau/Andy Lau_20.jpg
Andy Lau/Andy Lau_24.jpg
Keeping photo: Andy Lau/Andy Lau_24.jpg
Andy Lau/Andy Lau_25.jpg
Keeping photo: Andy Lau/Andy Lau_25.jpg
Andy Lau/Andy Lau_27.jpg
Keeping photo: Andy Lau/Andy Lau_27.jpg
Andy Lau/Andy Lau_28.jpg
Keeping photo: Andy Lau/Andy Lau_28.jpg
Andy Lau/Andy Lau_29.jpg
Keeping photo: Andy Lau/Andy Lau_29.jpg
Andy Lau/Andy Lau_32.jpg
Keeping photo: Andy Lau/Andy Lau_32.jpg
Andy Lau/Andy Lau_34.jpg
Keeping photo: Andy Lau/Andy Lau_34.jpg
Andy Lau/Andy Lau_35.jpg
Keeping photo: Andy Lau/Andy Lau_35.jpg
Andy Lau/Andy Lau_36.jpg


In [76]:
%%capture 
# Iterate through the objects in the bucket
for obj in response.get('Contents', [])[500:]:
    # Get the object key (file path)
    object_key = obj['Key']
    print(object_key)

    # Apply your function to the object
    if should_remove_photo(object_key, bucket_name):
        # Remove the file from the bucket
        s3.delete_object(Bucket=bucket_name, Key=object_key)
        
        print("Photo removed.")
    else:
        print("Keeping photo:", object_key)


KeyboardInterrupt: 

In [95]:
response = s3.list_objects_v2(Bucket=bucket_name)

print(type(response))
print(response.keys())
print(response['Contents'][980])

<class 'dict'>
dict_keys(['ResponseMetadata', 'IsTruncated', 'Contents', 'Name', 'Prefix', 'MaxKeys', 'EncodingType', 'KeyCount', 'NextContinuationToken'])
{'Key': 'Zhao Wei/Zhao Wei_7.jpg', 'LastModified': datetime.datetime(2023, 11, 25, 20, 0, 25, tzinfo=tzlocal()), 'ETag': '"c8da5a9b0b37707b243922b5ffbb2a63"', 'Size': 31533, 'StorageClass': 'STANDARD'}


In [98]:
response = s3.list_objects_v2(Bucket=bucket_name)
# Iterate through the objects in the bucket
for obj in response.get('Contents', [])[700:]:
    # Get the object key (file path)
    object_key = obj['Key']
    print(object_key)

    # Apply your function to the object
    if should_remove_photo(object_key, bucket_name):
        # Remove the file from the bucket
        s3.delete_object(Bucket=bucket_name, Key=object_key)
        
        print("Photo removed.")
    else:
        print("Keeping photo:", object_key)


Tong Liya/Tong Liya_38.jpg
Keeping photo: Tong Liya/Tong Liya_38.jpg
Tong Liya/Tong Liya_39.jpg
Keeping photo: Tong Liya/Tong Liya_39.jpg
Tong Liya/Tong Liya_40.jpg
Keeping photo: Tong Liya/Tong Liya_40.jpg
Tong Liya/Tong Liya_41.jpg
Keeping photo: Tong Liya/Tong Liya_41.jpg
Tong Liya/Tong Liya_42.jpg
Keeping photo: Tong Liya/Tong Liya_42.jpg
Tong Liya/Tong Liya_46.jpg
Keeping photo: Tong Liya/Tong Liya_46.jpg
Tong Liya/Tong Liya_47.jpg
Keeping photo: Tong Liya/Tong Liya_47.jpg
Tong Liya/Tong Liya_5.jpg
Keeping photo: Tong Liya/Tong Liya_5.jpg
Tong Liya/Tong Liya_50.jpg
Keeping photo: Tong Liya/Tong Liya_50.jpg
Tong Liya/Tong Liya_6.jpg
Keeping photo: Tong Liya/Tong Liya_6.jpg
Tong Liya/Tong Liya_7.jpg
Keeping photo: Tong Liya/Tong Liya_7.jpg
Tong Liya/Tong Liya_8.jpg
Keeping photo: Tong Liya/Tong Liya_8.jpg
Tong Liya/Tong Liya_9.jpg
Keeping photo: Tong Liya/Tong Liya_9.jpg
Wallace Huo/Wallace Huo_10.jpg
Keeping photo: Wallace Huo/Wallace Huo_10.jpg
Wallace Huo/Wallace Huo_13.jpg
Keepi

In [5]:
def delete_excess_objects(bucket_name, max_items_per_folder=10):
    s3 = boto3.client('s3')

    # List objects at the root level
    objects = s3.list_objects_v2(Bucket=bucket_name, Delimiter='/')

    # Iterate through each "folder" (common prefix)
    for common_prefix in objects.get('CommonPrefixes', []):
        folder_key = common_prefix['Prefix']
        print(f"Processing folder: {folder_key}")

        # List all objects in the folder
        folder_objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_key)

        # Sort objects by last modified time
        sorted_objects = sorted(folder_objects.get('Contents', []), key=lambda x: x['LastModified'])

        # Keep the last max_items_per_folder objects
        objects_to_keep = sorted_objects[-max_items_per_folder:]

        # Delete excess items in the folder
        for obj in sorted_objects[:-max_items_per_folder]:
            obj_key = obj['Key']
            print(f"Deleting excess item: {obj_key}")
            s3.delete_object(Bucket=bucket_name, Key=obj_key)


In [6]:
delete_excess_objects('chinese-celebs', max_items_per_folder=5)


Processing folder: Andy Lau/
Deleting excess item: Andy Lau/Andy Lau_34.jpg
Deleting excess item: Andy Lau/Andy Lau_35.jpg
Deleting excess item: Andy Lau/Andy Lau_36.jpg
Deleting excess item: Andy Lau/Andy Lau_37.jpg
Deleting excess item: Andy Lau/Andy Lau_43.jpg
Processing folder: Angelababy/
Deleting excess item: Angelababy/Angelababy_33.jpg
Deleting excess item: Angelababy/Angelababy_36.jpg
Deleting excess item: Angelababy/Angelababy_37.jpg
Deleting excess item: Angelababy/Angelababy_41.jpg
Deleting excess item: Angelababy/Angelababy_44.jpg
Processing folder: Chen Xuedong/
Deleting excess item: Chen Xuedong/Chen Xuedong_40.jpg
Deleting excess item: Chen Xuedong/Chen Xuedong_41.jpg
Deleting excess item: Chen Xuedong/Chen Xuedong_43.jpg
Deleting excess item: Chen Xuedong/Chen Xuedong_46.jpg
Deleting excess item: Chen Xuedong/Chen Xuedong_47.jpg
Processing folder: Chris Lee/
Deleting excess item: Chris Lee/chris_lee_1.jpg
Deleting excess item: Chris Lee/chris_lee_2.jpg
Deleting excess 